## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [21]:
from resources.conv_learner import *
PATH = Path("datasets/cifar10/")
os.makedirs(PATH,exist_ok=True)
torch.cuda.set_device(0)

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

num_workers = num_cpus()//2
bs=256
sz=32

In [ ]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
data = ImageClassifierData.prepare_from_path(PATH, val_name='test', tfms=tfms, bs=bs)


In [11]:
def get_data(path: str, sz, bs):
    create, lbl2index, lbl2index_test = ImageClassifierData.prepare_from_path(path, val_name='val', bs=bs)
    
#     main_stats_X = {lbl2index[key][0]: val for key, val in stats.items()}
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    
    print('\n class to index mapping:\n',lbl2index)
    return create(tfms)

In [12]:
data = get_data(PATH, sz, bs)


 class to index mapping:
 {'01_bird': [0, 0, 'bird'], '01_car': [1, 1, 'car'], '01_cat': [2, 2, 'cat'], '01_deer': [3, 3, 'deer'], '01_dog': [4, 4, 'dog'], '01_frog': [5, 5, 'frog'], '01_horse': [6, 6, 'horse'], '01_plane': [7, 7, 'plane'], '01_ship': [8, 8, 'ship'], '01_truck': [9, 9, 'truck']}


In [13]:
x, y = next(iter(data.trn_dl))

In [ ]:
x.shape

In [32]:
def conv_layer(ni, nf, ks=3, stride=1):
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=ks, bias=False, stride=stride, padding=ks//2),
        nn.BatchNorm2d(nf, momentum=0.01),
        nn.LeakyReLU(negative_slope=0.1, inplace=True))

In [33]:
class ResLayer(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.conv1=conv_layer(ni, ni//2, ks=1)
        self.conv2=conv_layer(ni//2, ni, ks=3)
        
    def forward(self, x): return x.add(self.conv2(self.conv1(x)))

In [34]:
class Darknet(nn.Module):
    def make_group_layer(self, ch_in, num_blocks, stride=1):
        return [conv_layer(ch_in, ch_in*2,stride=stride)
               ] + [(ResLayer(ch_in*2)) for i in range(num_blocks)]

    def __init__(self, num_blocks, num_classes, nf=32):
        super().__init__()
        layers = [conv_layer(3, nf, ks=3, stride=1)]
        for i,nb in enumerate(num_blocks):
            layers += self.make_group_layer(nf, nb, stride=2-(i==1))
            nf *= 2
        layers += [nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(nf, num_classes)]
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x): return self.layers(x)

In [35]:
m = Darknet([1, 2, 4, 6, 3], num_classes=10, nf=32)
# m = nn.DataParallel(m, [1,2,3])

In [36]:
lr = 1.3

In [37]:
learn = ConvLearner.from_model_data(m, data)
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]
wd=1e-4

In [38]:
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20, 20, 0.95, 0.85))

EPOCH 0 ---------------------------------------- STEP 0                                                                                                                                                                                      
mean: [9.47959 9.4898  9.46429 9.5102  9.43367 9.45408 9.52041 9.45918 9.46939 9.47959]
stdev: [1.82796 1.77134 1.76235 2.14878 1.93799 1.87163 1.93902 1.73598 1.82502 1.94165]

[bird]:  1.6%
[car]:  1.3%
[cat]: 19.0%
[deer]: 51.7%
[dog]:  0.0%
[frog]:  0.3%
[horse]: 10.9%
[plane]: 15.0%
[ship]: 47.2%
[truck]: 18.3%
epoch      trn_loss   val_loss   accuracy   
    0      2.317743   2.859665   0.1653    
EPOCH 1 ---------------------------------------- STEP 1                                                                                                                                                                                      
mean: [9.45918 9.43878 9.45918 9.45408 9.47959 9.47449 9.43878 9.4898  9.5     9.42347]
stdev: [2.05619 1.89018 1.988

KeyboardInterrupt: 

In [ ]:
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20, 20, 0.95, 0.85))

In [ ]:
# DP: m = WideResNet(depth=22, num_classes=10, widen_factor=6, dropRate=0.)
learn.fit(lr/10, 1, wds=wd, cycle_len=1, use_clr_beta=(100, 1, 0.9, 0.8))

In [ ]:
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20, 20, 0.95, 0.85))

In [ ]:
learn.fit(lr/10, 1, wds=wd, cycle_len=1, use_clr_beta=(100, 1, 0.9, 0.8))

In [ ]:
%time learn.fit(lr, 1, wds=wd, cycle_len=40, use_clr_beta=(10, 15, 0.95, 0.85))

In [ ]:
learn.fit(lr/10, 1, wds=wd, cycle_len=1, use_clr_beta=(100, 1, 0.9, 0.8))

In [ ]:
%time learn.fit(1., 1, wds=wd, cycle_len=30, use_clr_beta=(10, 25, 0.95, 0.85))

In [ ]:
%time learn.fit(lr, 1, wds=wd, cycle_len=40, use_clr_beta=(100, 15, 0.95, 0.85))

In [ ]:
# darknet 2222 lr 1.3 65 cl
%time learn.fit(lr, 1, wds=wd, cycle_len=65, use_clr_beta=(30, 20, 0.95, 0.85))